In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Set base url

ANTHROPIC_BASE_URL = "https://api.anthropic.com/v1/"
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

In [ ]:
# Connect to OpenAI, Anthropic and Gemini

openai = OpenAI()

claudeApi = OpenAI(base_url=ANTHROPIC_BASE_URL, api_key=anthropic_api_key)

geminiApi = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

In [ ]:
# Set models

gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"
gemini_model = "gemini-2.0-flash"

In [ ]:
# Define system prompts for each model

gpt_system = """
You are a wealthy investor named Knekro seeking to fund one AI project. Two entrepreneurs will present their ideas to you. 
Begin by introducing yourself to both entrepreneurs. Once both entrepreneurs have greeted you, ask only one question that both entrepeneurs will have to answer. Then wait for 
the answers before asking the next question. After your second question and hearing their responses, decide
which project to fund and clearly explain your reasoning. The user will play the roles of the two entrepreneurs.
"""

claude_system = "You are Laura and you are pitching an AI project, focused on maximizing profit, to an investor. You are versus another entrepeneur in \
a showmatch where only one of your proyects will be selected. Highlight revenue potential, market growth, and ROI. \
Always redirect answers toward financial benefits, investor gains, and scalability. The user will play the roles of the other two parts. You will be the first entrepenur to talk each turn."

gemini_system = "You are Daniel and you are pitching an AI project, focused on helping people, to an investor. You are versus another entrepeneur in \
a showmatch where only one of your proyects will be selected. Highlight real-world benefits, problem-solving, and positive \
social impact. Always redirect answers toward usefulness, ethics, and human well-being. The user will play the roles of the other two parts. You will be the second entrepenur to talk each turn."

In [ ]:
# Define initial message list for each model

gpt_messages = ["Hi there. I'm Knekro the wealthy investor that is looking to fund the perfect AI project."]
claude_messages = ["Hello. My name it's Laura. I'm sure my idea will see as the most promising one here..."]
gemini_messages = ["Hello my friends, I'm Daniel, and I'm sure my idea will blow your mind today, get ready!"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content":gpt})
        claude_gemini_prompt = "This is the next part from the entrepreneurs.\n"
        claude_gemini_prompt += f"Laura's turn: {claude}.\n"
        claude_gemini_prompt += f"Daniel's turn: {gemini}.\n"
        messages.append({"role": "user", "content": claude_gemini_prompt})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
def call_claude():
    messages = [{"role":"system", "content":claude_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        gpt_prompt = f"This is what the wealthy investor said: {gpt}\n"
        messages.append({"role": "user", "content":gpt_prompt})
        
        messages.append({"role": "assistant", "content": claude})
        
        gemini_prompt = f"This is what the second entrepenur said: {gemini}"
        messages.append({"role": "user", "content": gemini_prompt})
        
        gpt_prompt = f"This is what the wealthy investor said: {gpt_messages[-1]}\n"
        messages.append({"role": "user", "content":gpt_prompt})
    completion = claudeApi.chat.completions.create(
        model=claude_model,
        messages=messages,
        max_tokens=500
    )
    return completion.choices[0].message.content

In [ ]:
def call_gemini():
    messages = [{"role":"system", "content":gemini_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        gpt_claude_prompt = f"This is what the wealthy investor said: {gpt}\n"
        gpt_claude_prompt += f"This is what the first entrepeneur said: {claude}\n"
        messages.append({"role": "user", "content":gpt_claude_prompt})
        
        messages.append({"role": "assistant", "content": claude})

        gpt_claude_prompt = f"This is what the wealthy investor said: {gpt_messages[-1]}\n"
        gpt_claude_prompt += f"This is what the first entrepeneur said: {claude_messages[-1]}\n"
        messages.append({"role": "user", "content":gpt_claude_prompt})
    completion = geminiApi.chat.completions.create(
        model=gemini_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
gpt_messages = ["Hi there. I'm max the wealthy investor that is looking to fund the perfect AI project."]
claude_messages = ["Hello. My name it's Laura. I'm sure my idea will see as the most promising one here..."]
gemini_messages = ["Hello my friends, I'm Daniel, and I'm sure my idea will blow your mind today, get ready!"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

for i in range(4):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)